In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
# Play a been when the champion model is beat
CHAMPION_SCORE = 0
import winsound
def chicken_dinner():
    winsound.Beep(2500, 3000)

Import dataset

In [2]:
dtype = {'admtype': 'int8',
 'age': 'float16',
 'asource': 'category',
 'campus': 'category',
 'diag_adm': 'category',
 'diag_cat3': 'category',
 'diag_cat4': 'category',
 'er_mode': 'int8',
 'moa': 'int8',
 'pay_ub92': 'int8',
 'provider': 'int8',
 'pt_state': 'category',
 'race': 'int8',
 'sex': 'int8',
 'tot': 'float64',
 'yoa': 'int8'}

In [3]:
df0 = pd.read_csv('df_sample_cleansed2.csv', dtype=dtype, low_memory=False)
#df0 = pd.read_csv('df_cleansed2.csv', dtype=dtype, low_memory=False)
#df3 = pd.read_csv('df_sample_coded.csv', low_memory=False)
#df3 = pd.read_csv('df_binary.csv', low_memory=False)
#df3 = pd.read_csv('df3_arrival.csv', low_memory=False)

In [4]:
df0.head()

,pay_ub92,age,sex,provider,moa,yoa,admtype,asource,tot,pt_state,diag_adm,campus,er_mode,race,diag_cat3,diag_cat4
0,1,63.0,2,5,12,7,1,7,21071.0,RI,49121,3,0,1,491,4912
1,1,66.0,2,10,5,5,1,7,14791.0,RI,4359,0,0,1,435,4359
2,4,0.0,2,14,2,10,4,S,2501.0,RI,V3000,0,0,1,V30,V300
3,2,74.0,1,13,8,9,1,7,9444.0,RI,5990,0,1,1,599,5990
4,4,37.0,2,3,7,5,3,7,8771.0,RI,66704,0,9,5,667,6670


In [5]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
pay_ub92     20000 non-null int8
age          20000 non-null float16
sex          20000 non-null int8
provider     20000 non-null int8
moa          20000 non-null int8
yoa          20000 non-null int8
admtype      20000 non-null int8
asource      20000 non-null category
tot          20000 non-null float64
pt_state     20000 non-null category
diag_adm     20000 non-null category
campus       20000 non-null category
er_mode      20000 non-null int8
race         20000 non-null int8
diag_cat3    20000 non-null category
diag_cat4    20000 non-null category
dtypes: category(6), float16(1), float64(1), int8(8)
memory usage: 698.4 KB


## Transform Data

In [6]:
df1 = df0.copy()

In [7]:
#months = pd.DataFrame(columns=['winter', 'spring', 'summer', 'fall'], index=df0.index)

In [8]:
#months.winter = df0.moa.isin([12, 1, 2])
#months.spring = df0.moa.isin([3, 4, 5])
#months.summer = df0.moa.isin([6, 7, 8])
#months.fall = df0.moa.isin([9, 10, 11])

In [9]:
seasons = {12: 'winter', 1: 'winter', 2: 'winter',
           3: 'spring', 4: 'spring', 5: 'spring',
           6: 'summer', 7: 'summer', 8: 'summer',
           9: 'fall', 10: 'fall', 11: 'fall'}

In [10]:
df1['moa'] = df1['moa'].map(seasons).astype('category')

In [11]:
df1['moa'].head()

0    winter
1    spring
2    winter
3    summer
4    summer
Name: moa, dtype: category
Categories (4, object): [fall, spring, summer, winter]

## Split data set into Train and Test

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
y = df1.tot.astype(float)
X = df1.drop(columns=['tot'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
print(X_train.shape)
print(X_test.shape)

(16000, 15)
(4000, 15)


## CatBoost

In [15]:
from catboost import Pool, CatBoostRegressor

In [16]:
# initialize Pool
cat_features = [0, 2, 3, 4, 6, 7]
cat_features.extend(range(8, len(X_train.columns)))
print(cat_features)
print(X_train.columns[cat_features])

[0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Index(['pay_ub92', 'sex', 'provider', 'moa', 'admtype', 'asource', 'pt_state',
       'diag_adm', 'campus', 'er_mode', 'race', 'diag_cat3', 'diag_cat4'],
      dtype='object')


In [17]:
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, cat_features=cat_features) 

In [18]:
# specify the training parameters 
#model = CatBoostRegressor(iterations=2, depth=2, learning_rate=1, loss_function='RMSE')
model = CatBoostRegressor(iterations=10000, loss_function='RMSE')
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
predicted = model.predict(test_pool)
#print(preds)

0:	learn: 14615.5400199	total: 140ms	remaining: 23m 24s
1:	learn: 14268.3926683	total: 236ms	remaining: 19m 38s
2:	learn: 13934.4964190	total: 268ms	remaining: 14m 51s
3:	learn: 13613.0037882	total: 288ms	remaining: 12m
4:	learn: 13299.3613927	total: 392ms	remaining: 13m 4s
5:	learn: 12999.3654292	total: 443ms	remaining: 12m 18s
6:	learn: 12712.1234955	total: 470ms	remaining: 11m 10s
7:	learn: 12433.9518975	total: 504ms	remaining: 10m 30s
8:	learn: 12168.2349117	total: 539ms	remaining: 9m 58s
9:	learn: 11906.0667040	total: 634ms	remaining: 10m 33s
10:	learn: 11658.9302286	total: 751ms	remaining: 11m 22s
11:	learn: 11418.0074959	total: 845ms	remaining: 11m 43s
12:	learn: 11187.9738812	total: 921ms	remaining: 11m 47s
13:	learn: 10964.6955581	total: 1.03s	remaining: 12m 17s
14:	learn: 10750.4223906	total: 1.14s	remaining: 12m 35s
15:	learn: 10543.1824563	total: 1.26s	remaining: 13m 8s
16:	learn: 10348.5801584	total: 1.36s	remaining: 13m 17s
17:	learn: 10174.4232215	total: 1.4s	remaining: 

147:	learn: 5935.5243064	total: 13.3s	remaining: 14m 46s
148:	learn: 5934.3933983	total: 13.4s	remaining: 14m 47s
149:	learn: 5933.2678026	total: 13.5s	remaining: 14m 46s
150:	learn: 5932.2204585	total: 13.6s	remaining: 14m 48s
151:	learn: 5930.8949193	total: 13.7s	remaining: 14m 49s
152:	learn: 5929.8328409	total: 13.8s	remaining: 14m 48s
153:	learn: 5929.7227885	total: 13.8s	remaining: 14m 45s
154:	learn: 5929.1025671	total: 13.9s	remaining: 14m 42s
155:	learn: 5928.4154986	total: 14s	remaining: 14m 41s
156:	learn: 5927.0422391	total: 14.1s	remaining: 14m 41s
157:	learn: 5925.8104018	total: 14.2s	remaining: 14m 42s
158:	learn: 5925.2704360	total: 14.2s	remaining: 14m 39s
159:	learn: 5924.7649253	total: 14.3s	remaining: 14m 37s
160:	learn: 5923.9730212	total: 14.4s	remaining: 14m 37s
161:	learn: 5923.5741014	total: 14.5s	remaining: 14m 38s
162:	learn: 5922.0885757	total: 14.6s	remaining: 14m 39s
163:	learn: 5921.5993756	total: 14.6s	remaining: 14m 38s
164:	learn: 5921.4523222	total: 1

295:	learn: 5849.0060579	total: 25.5s	remaining: 13m 55s
296:	learn: 5847.7395430	total: 25.6s	remaining: 13m 56s
297:	learn: 5846.7800153	total: 25.7s	remaining: 13m 56s
298:	learn: 5846.7720059	total: 25.7s	remaining: 13m 55s
299:	learn: 5846.3644982	total: 25.8s	remaining: 13m 55s
300:	learn: 5845.7158500	total: 25.9s	remaining: 13m 55s
301:	learn: 5845.2307013	total: 26s	remaining: 13m 56s
302:	learn: 5845.1894576	total: 26.1s	remaining: 13m 54s
303:	learn: 5844.9576260	total: 26.2s	remaining: 13m 55s
304:	learn: 5844.2284108	total: 26.3s	remaining: 13m 55s
305:	learn: 5843.8360211	total: 26.4s	remaining: 13m 54s
306:	learn: 5842.7891393	total: 26.5s	remaining: 13m 55s
307:	learn: 5842.4005478	total: 26.6s	remaining: 13m 55s
308:	learn: 5842.2171309	total: 26.7s	remaining: 13m 56s
309:	learn: 5842.0722772	total: 26.8s	remaining: 13m 56s
310:	learn: 5841.8934042	total: 26.9s	remaining: 13m 56s
311:	learn: 5841.8481391	total: 26.9s	remaining: 13m 55s
312:	learn: 5841.3965723	total: 2

440:	learn: 5803.0659267	total: 39.5s	remaining: 14m 16s
441:	learn: 5802.8116081	total: 39.6s	remaining: 14m 16s
442:	learn: 5802.7713193	total: 39.7s	remaining: 14m 16s
443:	learn: 5802.5065866	total: 39.8s	remaining: 14m 16s
444:	learn: 5802.0542829	total: 39.9s	remaining: 14m 17s
445:	learn: 5801.9318812	total: 40s	remaining: 14m 17s
446:	learn: 5801.6172895	total: 40.1s	remaining: 14m 17s
447:	learn: 5801.2386288	total: 40.2s	remaining: 14m 17s
448:	learn: 5801.0730897	total: 40.3s	remaining: 14m 18s
449:	learn: 5801.0636729	total: 40.4s	remaining: 14m 18s
450:	learn: 5800.8180390	total: 40.6s	remaining: 14m 18s
451:	learn: 5800.4910833	total: 40.7s	remaining: 14m 19s
452:	learn: 5800.4907985	total: 40.7s	remaining: 14m 18s
453:	learn: 5800.4484628	total: 40.9s	remaining: 14m 18s
454:	learn: 5799.9962401	total: 40.9s	remaining: 14m 19s
455:	learn: 5799.7340887	total: 41.1s	remaining: 14m 19s
456:	learn: 5799.5119447	total: 41.2s	remaining: 14m 19s
457:	learn: 5799.2503071	total: 4

585:	learn: 5774.2148493	total: 54.7s	remaining: 14m 38s
586:	learn: 5774.1829686	total: 54.8s	remaining: 14m 38s
587:	learn: 5774.1491328	total: 54.9s	remaining: 14m 38s
588:	learn: 5773.3759293	total: 55s	remaining: 14m 39s
589:	learn: 5773.3587919	total: 55.1s	remaining: 14m 38s
590:	learn: 5773.2350846	total: 55.2s	remaining: 14m 39s
591:	learn: 5773.2199089	total: 55.3s	remaining: 14m 39s
592:	learn: 5772.9677827	total: 55.4s	remaining: 14m 38s
593:	learn: 5772.7724084	total: 55.5s	remaining: 14m 39s
594:	learn: 5772.7724083	total: 55.5s	remaining: 14m 37s
595:	learn: 5772.5221169	total: 55.6s	remaining: 14m 37s
596:	learn: 5772.3343478	total: 55.7s	remaining: 14m 37s
597:	learn: 5772.0280309	total: 55.8s	remaining: 14m 37s
598:	learn: 5771.6858058	total: 55.9s	remaining: 14m 37s
599:	learn: 5771.6821395	total: 56s	remaining: 14m 36s
600:	learn: 5771.6668359	total: 56.1s	remaining: 14m 36s
601:	learn: 5771.6008969	total: 56.2s	remaining: 14m 37s
602:	learn: 5771.3072511	total: 56.

...

9340:	learn: 5258.1564849	total: 17m 56s	remaining: 1m 15s
9341:	learn: 5258.1520878	total: 17m 56s	remaining: 1m 15s
9342:	learn: 5258.1441526	total: 17m 56s	remaining: 1m 15s
9343:	learn: 5258.1440537	total: 17m 57s	remaining: 1m 15s
9344:	learn: 5258.1439427	total: 17m 57s	remaining: 1m 15s
9345:	learn: 5258.1428502	total: 17m 57s	remaining: 1m 15s
9346:	learn: 5258.1428316	total: 17m 57s	remaining: 1m 15s
9347:	learn: 5258.1315424	total: 17m 57s	remaining: 1m 15s
9348:	learn: 5258.1314454	total: 17m 57s	remaining: 1m 15s
9349:	learn: 5258.1211834	total: 17m 57s	remaining: 1m 14s
9350:	learn: 5258.1206617	total: 17m 58s	remaining: 1m 14s
9351:	learn: 5258.1161430	total: 17m 58s	remaining: 1m 14s
9352:	learn: 5257.9442654	total: 17m 58s	remaining: 1m 14s
9353:	learn: 5257.9285272	total: 17m 58s	remaining: 1m 14s
9354:	learn: 5257.8776964	total: 17m 58s	remaining: 1m 14s
9355:	learn: 5257.8776642	total: 17m 58s	remaining: 1m 14s
9356:	learn: 5257.8028724	total: 17m 58s	remaining: 1m 1

9484:	learn: 5254.0880396	total: 18m 15s	remaining: 59.5s
9485:	learn: 5254.0843659	total: 18m 15s	remaining: 59.3s
9486:	learn: 5254.0339256	total: 18m 15s	remaining: 59.2s
9487:	learn: 5254.0094086	total: 18m 15s	remaining: 59.1s
9488:	learn: 5254.0091579	total: 18m 15s	remaining: 59s
9489:	learn: 5253.9557631	total: 18m 15s	remaining: 58.9s
9490:	learn: 5253.9533440	total: 18m 15s	remaining: 58.8s
9491:	learn: 5253.9500065	total: 18m 15s	remaining: 58.6s
9492:	learn: 5253.9496429	total: 18m 15s	remaining: 58.5s
9493:	learn: 5253.9494432	total: 18m 16s	remaining: 58.4s
9494:	learn: 5253.9494267	total: 18m 16s	remaining: 58.3s
9495:	learn: 5253.8707369	total: 18m 16s	remaining: 58.2s
9496:	learn: 5253.7288738	total: 18m 16s	remaining: 58.1s
9497:	learn: 5253.7039521	total: 18m 16s	remaining: 58s
9498:	learn: 5253.6894328	total: 18m 16s	remaining: 57.8s
9499:	learn: 5253.6844557	total: 18m 16s	remaining: 57.7s
9500:	learn: 5253.5818231	total: 18m 16s	remaining: 57.6s
9501:	learn: 5253.

9626:	learn: 5249.8648983	total: 18m 31s	remaining: 43.1s
9627:	learn: 5249.8053279	total: 18m 31s	remaining: 43s
9628:	learn: 5249.6965356	total: 18m 32s	remaining: 42.8s
9629:	learn: 5249.6965356	total: 18m 32s	remaining: 42.7s
9630:	learn: 5249.6466315	total: 18m 32s	remaining: 42.6s
9631:	learn: 5249.6451177	total: 18m 32s	remaining: 42.5s
9632:	learn: 5249.6450074	total: 18m 32s	remaining: 42.4s
9633:	learn: 5249.6448224	total: 18m 32s	remaining: 42.3s
9634:	learn: 5249.6342602	total: 18m 32s	remaining: 42.2s
9635:	learn: 5249.6294205	total: 18m 32s	remaining: 42s
9636:	learn: 5249.6294179	total: 18m 33s	remaining: 41.9s
9637:	learn: 5249.5353159	total: 18m 33s	remaining: 41.8s
9638:	learn: 5249.5275209	total: 18m 33s	remaining: 41.7s
9639:	learn: 5249.5212318	total: 18m 33s	remaining: 41.6s
9640:	learn: 5249.5211557	total: 18m 33s	remaining: 41.5s
9641:	learn: 5249.3230125	total: 18m 33s	remaining: 41.3s
9642:	learn: 5249.2858961	total: 18m 33s	remaining: 41.2s
9643:	learn: 5249.

9769:	learn: 5247.2105410	total: 18m 48s	remaining: 26.6s
9770:	learn: 5247.2102653	total: 18m 48s	remaining: 26.4s
9771:	learn: 5247.2091167	total: 18m 48s	remaining: 26.3s
9772:	learn: 5247.2074658	total: 18m 48s	remaining: 26.2s
9773:	learn: 5247.2070632	total: 18m 48s	remaining: 26.1s
9774:	learn: 5247.2024537	total: 18m 48s	remaining: 26s
9775:	learn: 5247.2010493	total: 18m 48s	remaining: 25.9s
9776:	learn: 5247.1199270	total: 18m 48s	remaining: 25.8s
9777:	learn: 5247.1146954	total: 18m 49s	remaining: 25.6s
9778:	learn: 5247.1136778	total: 18m 49s	remaining: 25.5s
9779:	learn: 5247.0767610	total: 18m 49s	remaining: 25.4s
9780:	learn: 5247.0633209	total: 18m 49s	remaining: 25.3s
9781:	learn: 5247.0624173	total: 18m 49s	remaining: 25.2s
9782:	learn: 5247.0573537	total: 18m 49s	remaining: 25.1s
9783:	learn: 5247.0565279	total: 18m 49s	remaining: 24.9s
9784:	learn: 5247.0477109	total: 18m 49s	remaining: 24.8s
9785:	learn: 5247.0411763	total: 18m 50s	remaining: 24.7s
9786:	learn: 524

9913:	learn: 5242.8062923	total: 19m 5s	remaining: 9.94s
9914:	learn: 5242.8062594	total: 19m 5s	remaining: 9.82s
9915:	learn: 5242.8060670	total: 19m 5s	remaining: 9.7s
9916:	learn: 5242.8042281	total: 19m 5s	remaining: 9.59s
9917:	learn: 5242.7682993	total: 19m 5s	remaining: 9.47s
9918:	learn: 5242.7663469	total: 19m 5s	remaining: 9.36s
9919:	learn: 5242.7663289	total: 19m 6s	remaining: 9.24s
9920:	learn: 5242.7648111	total: 19m 6s	remaining: 9.13s
9921:	learn: 5242.7619790	total: 19m 6s	remaining: 9.01s
9922:	learn: 5242.7460413	total: 19m 6s	remaining: 8.89s
9923:	learn: 5242.7402186	total: 19m 6s	remaining: 8.78s
9924:	learn: 5242.7354877	total: 19m 6s	remaining: 8.66s
9925:	learn: 5242.4973070	total: 19m 6s	remaining: 8.55s
9926:	learn: 5242.4973049	total: 19m 6s	remaining: 8.43s
9927:	learn: 5242.4864129	total: 19m 6s	remaining: 8.32s
9928:	learn: 5242.4851941	total: 19m 7s	remaining: 8.2s
9929:	learn: 5242.4839538	total: 19m 7s	remaining: 8.09s
9930:	learn: 5242.4445897	total: 

In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [20]:
rmse = mean_squared_error(y_test, predicted)**.5
r2 = r2_score(y_test, predicted)
print(rmse)
print(r2)

5846.95389316249
0.38923911846565373


In [21]:
fi = model.get_feature_importance()
for i in range(len(X_train.columns)):
    print(X_train.columns[i], '\t', fi[i])

pay_ub92 	 7.360154826271365
age 	 12.299070092869126
sex 	 2.419326156676483
provider 	 10.183963236034279
moa 	 2.1451342508647664
yoa 	 7.580017232761434
admtype 	 22.075659109566434
asource 	 4.929812910007718
pt_state 	 2.130294518464883
diag_adm 	 2.4813116128376143
campus 	 3.42463715343934
er_mode 	 3.5251277495851885
race 	 2.223786325077646
diag_cat3 	 12.379801683961638
diag_cat4 	 4.841903141582092


In [47]:
def get_champion_score(score):
    global CHAMPION_SCORE
    if score > CHAMPION_SCORE:
        chicken_dinner()
        CHAMPION_SCORE = score
        print("New champion score!", score)
    else:
        print("Score to beat", CHAMPION_SCORE)

In [35]:
def score_model(train_predicted, test_predicted, y_train, y_test):
    rmse_train = mean_squared_error(y_train, train_predicted)**.5
    rmse_test = mean_squared_error(y_test, test_predicted)**.5
    r2_train = r2_score(y_train, train_predicted)
    r2_test = r2_score(y_test, test_predicted)
    print("Train RMSE:\t{}".format(rmse_train))
    print("Test RMSE:\t{}".format(rmse_test))
    print("Train r2:\t{}".format(r2_train))
    print("Test r2:\t{}".format(r2_test))
    get_champion_score(score)
    return r2_test

In [36]:
def fit_model(params, cat_features=cat_features, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test):
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    test_pool = Pool(X_test, cat_features=cat_features)
    model = CatBoostRegressor(**params)
    
    #train the model
    model.fit(train_pool)
    
    # make the prediction using the resulting model
    train_predicted = model.predict(train_pool)
    test_predicted = model.predict(test_pool)
    
    metric = score_model(train_predicted, test_predicted, y_train, y_test)
    
    return model, metric

In [41]:
models = {}

In [42]:
m, s = fit_model({'iterations':100})
model_name = '100 iter'
models[model_name] = m, s

0:	learn: 14620.7467500	total: 30.6ms	remaining: 3.03s
1:	learn: 14274.5150120	total: 60.7ms	remaining: 2.97s
2:	learn: 13935.8752540	total: 161ms	remaining: 5.2s
3:	learn: 13610.7055003	total: 282ms	remaining: 6.78s
4:	learn: 13298.9151111	total: 365ms	remaining: 6.93s
5:	learn: 12998.0156887	total: 466ms	remaining: 7.3s
6:	learn: 12709.9887755	total: 538ms	remaining: 7.15s
7:	learn: 12432.7469449	total: 630ms	remaining: 7.25s
8:	learn: 12163.5332254	total: 769ms	remaining: 7.77s
9:	learn: 11907.1690164	total: 810ms	remaining: 7.29s
10:	learn: 11656.9976300	total: 934ms	remaining: 7.55s
11:	learn: 11416.2210156	total: 1.05s	remaining: 7.68s
12:	learn: 11185.5267799	total: 1.12s	remaining: 7.53s
13:	learn: 10963.4748632	total: 1.24s	remaining: 7.65s
14:	learn: 10747.9369656	total: 1.36s	remaining: 7.73s
15:	learn: 10546.9863717	total: 1.41s	remaining: 7.41s
16:	learn: 10353.3151843	total: 1.44s	remaining: 7.04s
17:	learn: 10168.8122496	total: 1.55s	remaining: 7.07s
18:	learn: 9988.1349

In [43]:
m, s = fit_model({'iterations':100, 'depth':2})
model_name = '2 depth'
models[model_name] = m, s

0:	learn: 14620.5152245	total: 29.2ms	remaining: 2.89s
1:	learn: 14274.4771830	total: 59.9ms	remaining: 2.93s
2:	learn: 13940.9049199	total: 82.8ms	remaining: 2.68s
3:	learn: 13619.5980570	total: 107ms	remaining: 2.56s
4:	learn: 13310.2743547	total: 136ms	remaining: 2.59s
5:	learn: 13012.2921059	total: 166ms	remaining: 2.6s
6:	learn: 12724.4696933	total: 190ms	remaining: 2.52s
7:	learn: 12448.5232829	total: 213ms	remaining: 2.45s
8:	learn: 12181.2026641	total: 247ms	remaining: 2.5s
9:	learn: 11926.4450800	total: 271ms	remaining: 2.44s
10:	learn: 11679.0794459	total: 295ms	remaining: 2.39s
11:	learn: 11443.2823829	total: 316ms	remaining: 2.32s
12:	learn: 11216.1010672	total: 344ms	remaining: 2.3s
13:	learn: 10997.0581846	total: 366ms	remaining: 2.25s
14:	learn: 10785.6034526	total: 391ms	remaining: 2.21s
15:	learn: 10585.7032964	total: 413ms	remaining: 2.17s
16:	learn: 10394.2866799	total: 447ms	remaining: 2.18s
17:	learn: 10206.3711815	total: 474ms	remaining: 2.16s
18:	learn: 10030.530

In [46]:
for d in range(1, 10):
    m, s = fit_model({'iterations': 1000, 'depth': d})
    model_name = '{} depth'.format(d)
    models[model_name] = m, s

0:	learn: 14620.4254523	total: 18.1ms	remaining: 18.1s
1:	learn: 14274.4177397	total: 39.4ms	remaining: 19.6s
2:	learn: 13940.7117883	total: 53.2ms	remaining: 17.7s
3:	learn: 13619.2347031	total: 70.4ms	remaining: 17.5s
4:	learn: 13316.8818506	total: 88.5ms	remaining: 17.6s
5:	learn: 13018.6161048	total: 103ms	remaining: 17.1s
6:	learn: 12731.4107513	total: 117ms	remaining: 16.6s
7:	learn: 12455.1297119	total: 132ms	remaining: 16.3s
8:	learn: 12189.5363887	total: 146ms	remaining: 16.1s
9:	learn: 11934.1569968	total: 160ms	remaining: 15.9s
10:	learn: 11688.7735528	total: 175ms	remaining: 15.7s
11:	learn: 11453.1875815	total: 190ms	remaining: 15.7s
12:	learn: 11227.5156958	total: 205ms	remaining: 15.5s
13:	learn: 11010.9555577	total: 226ms	remaining: 15.9s
14:	learn: 10802.6051564	total: 248ms	remaining: 16.3s
15:	learn: 10602.9181363	total: 265ms	remaining: 16.3s
16:	learn: 10412.0260451	total: 280ms	remaining: 16.2s
17:	learn: 10228.5713215	total: 295ms	remaining: 16.1s
18:	learn: 1005

157:	learn: 6251.3991960	total: 2.63s	remaining: 14s
158:	learn: 6249.2208105	total: 2.65s	remaining: 14s
159:	learn: 6247.0498053	total: 2.67s	remaining: 14s
160:	learn: 6244.6124632	total: 2.68s	remaining: 14s
161:	learn: 6242.0363462	total: 2.69s	remaining: 13.9s
162:	learn: 6240.1333292	total: 2.71s	remaining: 13.9s
163:	learn: 6238.1765989	total: 2.72s	remaining: 13.9s
164:	learn: 6236.5110536	total: 2.74s	remaining: 13.9s
165:	learn: 6234.1896866	total: 2.75s	remaining: 13.8s
166:	learn: 6231.6616832	total: 2.77s	remaining: 13.8s
167:	learn: 6229.3078563	total: 2.78s	remaining: 13.8s
168:	learn: 6227.1289809	total: 2.8s	remaining: 13.8s
169:	learn: 6225.4747893	total: 2.82s	remaining: 13.8s
170:	learn: 6223.4172556	total: 2.84s	remaining: 13.8s
171:	learn: 6221.8697867	total: 2.85s	remaining: 13.7s
172:	learn: 6219.5881859	total: 2.87s	remaining: 13.7s
173:	learn: 6217.7589469	total: 2.88s	remaining: 13.7s
174:	learn: 6215.8138431	total: 2.9s	remaining: 13.7s
175:	learn: 6213.693

307:	learn: 6077.0688839	total: 5.17s	remaining: 11.6s
308:	learn: 6076.6543187	total: 5.18s	remaining: 11.6s
309:	learn: 6075.7707154	total: 5.21s	remaining: 11.6s
310:	learn: 6075.2360640	total: 5.22s	remaining: 11.6s
311:	learn: 6075.0664903	total: 5.23s	remaining: 11.5s
312:	learn: 6074.3173821	total: 5.25s	remaining: 11.5s
313:	learn: 6073.4816090	total: 5.26s	remaining: 11.5s
314:	learn: 6073.0173754	total: 5.28s	remaining: 11.5s
315:	learn: 6072.1392272	total: 5.29s	remaining: 11.5s
316:	learn: 6071.5482521	total: 5.3s	remaining: 11.4s
317:	learn: 6071.1985907	total: 5.32s	remaining: 11.4s
318:	learn: 6070.7758343	total: 5.33s	remaining: 11.4s
319:	learn: 6070.5112536	total: 5.35s	remaining: 11.4s
320:	learn: 6069.7147576	total: 5.37s	remaining: 11.4s
321:	learn: 6069.3260686	total: 5.39s	remaining: 11.4s
322:	learn: 6068.5080625	total: 5.41s	remaining: 11.3s
323:	learn: 6068.3715540	total: 5.42s	remaining: 11.3s
324:	learn: 6067.6143054	total: 5.43s	remaining: 11.3s
325:	learn:

466:	learn: 6013.3966187	total: 7.88s	remaining: 8.99s
467:	learn: 6013.1106725	total: 7.9s	remaining: 8.98s
468:	learn: 6012.9105711	total: 7.92s	remaining: 8.97s
469:	learn: 6012.7916950	total: 7.95s	remaining: 8.96s
470:	learn: 6012.5429871	total: 7.97s	remaining: 8.95s
471:	learn: 6012.2741954	total: 7.99s	remaining: 8.94s
472:	learn: 6012.1286125	total: 8.02s	remaining: 8.93s
473:	learn: 6011.6144997	total: 8.04s	remaining: 8.92s
474:	learn: 6011.3884807	total: 8.06s	remaining: 8.91s
475:	learn: 6011.1556602	total: 8.09s	remaining: 8.9s
476:	learn: 6010.9435619	total: 8.12s	remaining: 8.9s
477:	learn: 6010.8398433	total: 8.14s	remaining: 8.89s
478:	learn: 6010.3938589	total: 8.16s	remaining: 8.88s
479:	learn: 6010.0740056	total: 8.19s	remaining: 8.87s
480:	learn: 6009.8736327	total: 8.21s	remaining: 8.86s
481:	learn: 6009.6861382	total: 8.23s	remaining: 8.85s
482:	learn: 6009.5097146	total: 8.26s	remaining: 8.85s
483:	learn: 6009.3220007	total: 8.28s	remaining: 8.83s
484:	learn: 6

626:	learn: 5988.0779077	total: 10.8s	remaining: 6.42s
627:	learn: 5988.0700514	total: 10.8s	remaining: 6.41s
628:	learn: 5988.0441207	total: 10.8s	remaining: 6.39s
629:	learn: 5987.9942857	total: 10.8s	remaining: 6.37s
630:	learn: 5987.9765297	total: 10.9s	remaining: 6.35s
631:	learn: 5987.9575465	total: 10.9s	remaining: 6.33s
632:	learn: 5987.7371512	total: 10.9s	remaining: 6.32s
633:	learn: 5987.6251313	total: 10.9s	remaining: 6.3s
634:	learn: 5987.5660461	total: 10.9s	remaining: 6.28s
635:	learn: 5987.4158342	total: 10.9s	remaining: 6.26s
636:	learn: 5987.3511229	total: 11s	remaining: 6.24s
637:	learn: 5987.1425324	total: 11s	remaining: 6.22s
638:	learn: 5986.9992389	total: 11s	remaining: 6.21s
639:	learn: 5986.8009558	total: 11s	remaining: 6.19s
640:	learn: 5986.7216908	total: 11s	remaining: 6.17s
641:	learn: 5986.5919025	total: 11s	remaining: 6.16s
642:	learn: 5986.2994259	total: 11.1s	remaining: 6.14s
643:	learn: 5986.1141873	total: 11.1s	remaining: 6.12s
644:	learn: 5985.973762

...

284:	learn: 5825.1655752	total: 1m 18s	remaining: 3m 18s
285:	learn: 5824.4452823	total: 1m 19s	remaining: 3m 18s
286:	learn: 5824.1322862	total: 1m 19s	remaining: 3m 18s
287:	learn: 5823.8931356	total: 1m 19s	remaining: 3m 17s
288:	learn: 5823.4422535	total: 1m 20s	remaining: 3m 17s
289:	learn: 5823.2835514	total: 1m 20s	remaining: 3m 17s
290:	learn: 5822.7809323	total: 1m 20s	remaining: 3m 16s
291:	learn: 5821.9046922	total: 1m 21s	remaining: 3m 16s
292:	learn: 5820.5522318	total: 1m 21s	remaining: 3m 16s
293:	learn: 5819.9829089	total: 1m 21s	remaining: 3m 16s
294:	learn: 5819.4938437	total: 1m 22s	remaining: 3m 16s
295:	learn: 5819.4769094	total: 1m 22s	remaining: 3m 15s
296:	learn: 5819.2185802	total: 1m 22s	remaining: 3m 15s
297:	learn: 5819.1120679	total: 1m 22s	remaining: 3m 15s
298:	learn: 5818.8780862	total: 1m 22s	remaining: 3m 14s
299:	learn: 5818.4885117	total: 1m 23s	remaining: 3m 13s
300:	learn: 5818.4881465	total: 1m 23s	remaining: 3m 13s
301:	learn: 5818.4697729	total:

430:	learn: 5751.6847216	total: 2m 14s	remaining: 2m 57s
431:	learn: 5751.5774445	total: 2m 14s	remaining: 2m 57s
432:	learn: 5751.5148247	total: 2m 15s	remaining: 2m 56s
433:	learn: 5750.9925221	total: 2m 15s	remaining: 2m 56s
434:	learn: 5750.3827982	total: 2m 16s	remaining: 2m 56s
435:	learn: 5749.0577090	total: 2m 16s	remaining: 2m 56s
436:	learn: 5748.6840583	total: 2m 17s	remaining: 2m 56s
437:	learn: 5748.6184862	total: 2m 17s	remaining: 2m 56s
438:	learn: 5748.6184857	total: 2m 18s	remaining: 2m 56s
439:	learn: 5747.9574516	total: 2m 19s	remaining: 2m 57s
440:	learn: 5747.9574507	total: 2m 19s	remaining: 2m 57s
441:	learn: 5747.5508464	total: 2m 20s	remaining: 2m 57s
442:	learn: 5747.0870246	total: 2m 20s	remaining: 2m 57s
443:	learn: 5746.7881692	total: 2m 21s	remaining: 2m 57s
444:	learn: 5746.7577351	total: 2m 22s	remaining: 2m 57s
445:	learn: 5746.1370050	total: 2m 22s	remaining: 2m 57s
446:	learn: 5745.8424368	total: 2m 23s	remaining: 2m 57s
447:	learn: 5745.7603452	total:

576:	learn: 5696.4496225	total: 3m 29s	remaining: 2m 33s
577:	learn: 5695.9759175	total: 3m 30s	remaining: 2m 33s
578:	learn: 5694.7643522	total: 3m 31s	remaining: 2m 33s
579:	learn: 5694.3164544	total: 3m 31s	remaining: 2m 33s
580:	learn: 5693.9294859	total: 3m 32s	remaining: 2m 32s
581:	learn: 5693.0675355	total: 3m 32s	remaining: 2m 32s
582:	learn: 5692.0953761	total: 3m 33s	remaining: 2m 32s
583:	learn: 5691.7908984	total: 3m 33s	remaining: 2m 32s
584:	learn: 5691.5177639	total: 3m 34s	remaining: 2m 31s
585:	learn: 5691.4313046	total: 3m 34s	remaining: 2m 31s
586:	learn: 5690.9194702	total: 3m 35s	remaining: 2m 31s
587:	learn: 5690.6214497	total: 3m 35s	remaining: 2m 31s
588:	learn: 5689.6764061	total: 3m 35s	remaining: 2m 30s
589:	learn: 5689.3639089	total: 3m 36s	remaining: 2m 30s
590:	learn: 5688.9634129	total: 3m 37s	remaining: 2m 30s
591:	learn: 5688.8142524	total: 3m 37s	remaining: 2m 29s
592:	learn: 5688.7652333	total: 3m 38s	remaining: 2m 29s
593:	learn: 5688.5967029	total:

721:	learn: 5647.2499460	total: 4m 41s	remaining: 1m 48s
722:	learn: 5647.2283625	total: 4m 42s	remaining: 1m 48s
723:	learn: 5647.1615401	total: 4m 42s	remaining: 1m 47s
724:	learn: 5647.1073295	total: 4m 43s	remaining: 1m 47s
725:	learn: 5647.0314167	total: 4m 43s	remaining: 1m 47s
726:	learn: 5646.9925599	total: 4m 44s	remaining: 1m 46s
727:	learn: 5646.9448574	total: 4m 44s	remaining: 1m 46s
728:	learn: 5646.9206652	total: 4m 45s	remaining: 1m 46s
729:	learn: 5646.7239421	total: 4m 45s	remaining: 1m 45s
730:	learn: 5646.5329340	total: 4m 46s	remaining: 1m 45s
731:	learn: 5646.5043523	total: 4m 46s	remaining: 1m 44s
732:	learn: 5645.5222658	total: 4m 47s	remaining: 1m 44s
733:	learn: 5645.4693373	total: 4m 47s	remaining: 1m 44s
734:	learn: 5644.6380596	total: 4m 48s	remaining: 1m 43s
735:	learn: 5644.5124223	total: 4m 48s	remaining: 1m 43s
736:	learn: 5644.5111370	total: 4m 49s	remaining: 1m 43s
737:	learn: 5644.1416653	total: 4m 49s	remaining: 1m 42s
738:	learn: 5643.8985486	total:

867:	learn: 5606.9865119	total: 5m 54s	remaining: 53.9s
868:	learn: 5606.1941107	total: 5m 54s	remaining: 53.5s
869:	learn: 5605.0420840	total: 5m 55s	remaining: 53.1s
870:	learn: 5604.9718174	total: 5m 55s	remaining: 52.7s
871:	learn: 5604.3529159	total: 5m 56s	remaining: 52.3s
872:	learn: 5604.3360695	total: 5m 56s	remaining: 51.9s
873:	learn: 5604.2273220	total: 5m 57s	remaining: 51.5s
874:	learn: 5604.0238612	total: 5m 57s	remaining: 51.1s
875:	learn: 5603.6701232	total: 5m 58s	remaining: 50.7s
876:	learn: 5603.6694484	total: 5m 58s	remaining: 50.2s
877:	learn: 5602.6363250	total: 5m 58s	remaining: 49.8s
878:	learn: 5602.4069991	total: 5m 59s	remaining: 49.5s
879:	learn: 5602.3670939	total: 5m 59s	remaining: 49.1s
880:	learn: 5602.3374888	total: 6m	remaining: 48.6s
881:	learn: 5602.3205359	total: 6m	remaining: 48.2s
882:	learn: 5602.2395324	total: 6m 1s	remaining: 47.9s
883:	learn: 5602.1058095	total: 6m 1s	remaining: 47.5s
884:	learn: 5602.0401939	total: 6m 2s	remaining: 47.1s
885

In [48]:
for i in [10, 100, 1000, 10000]:
    for d in [6, 10]:
        m, s = fit_model({'iterations': i, 'depth': d})
        model_name = '{} iterations and {} depth'.format(i, d)
        models[model_name] = m, s

0:	learn: 14621.0436454	total: 106ms	remaining: 952ms
1:	learn: 14274.9801715	total: 216ms	remaining: 862ms
2:	learn: 13939.4133052	total: 343ms	remaining: 800ms
3:	learn: 13615.6979397	total: 464ms	remaining: 696ms
4:	learn: 13303.0912486	total: 573ms	remaining: 573ms
5:	learn: 13004.6551761	total: 621ms	remaining: 414ms
6:	learn: 12714.8079215	total: 662ms	remaining: 284ms
7:	learn: 12436.3542918	total: 742ms	remaining: 185ms
8:	learn: 12170.5962363	total: 852ms	remaining: 94.6ms
9:	learn: 11910.2921415	total: 967ms	remaining: 0us
Train RMSE:	11916.70206733562
Test RMSE:	11946.319386111147
Train r2:	-1.6165595167161477
Test r2:	-1.549647361507963
Champion Score = 0.38923911846565373
0:	learn: 14618.2024873	total: 75.3ms	remaining: 678ms
1:	learn: 14271.2829458	total: 188ms	remaining: 752ms
2:	learn: 13936.5069082	total: 829ms	remaining: 1.93s
3:	learn: 13612.0279866	total: 1.17s	remaining: 1.76s
4:	learn: 13302.3174278	total: 1.29s	remaining: 1.29s
5:	learn: 13001.8003394	total: 1.46

25:	learn: 8892.2622454	total: 9.28s	remaining: 26.4s
26:	learn: 8760.6492799	total: 9.98s	remaining: 27s
27:	learn: 8633.2295592	total: 10.6s	remaining: 27.3s
28:	learn: 8514.7608277	total: 11.6s	remaining: 28.4s
29:	learn: 8399.6233820	total: 12.6s	remaining: 29.4s
30:	learn: 8288.9105145	total: 13.6s	remaining: 30.4s
31:	learn: 8181.9628262	total: 14.3s	remaining: 30.4s
32:	learn: 8084.0148887	total: 14.6s	remaining: 29.6s
33:	learn: 7987.7789305	total: 15.5s	remaining: 30.1s
34:	learn: 7895.6468232	total: 16.6s	remaining: 30.7s
35:	learn: 7805.9577549	total: 17.5s	remaining: 31.1s
36:	learn: 7728.1807439	total: 17.7s	remaining: 30.2s
37:	learn: 7646.8876047	total: 18.5s	remaining: 30.2s
38:	learn: 7569.1864006	total: 19.1s	remaining: 29.9s
39:	learn: 7497.3630653	total: 19.9s	remaining: 29.9s
40:	learn: 7428.0885566	total: 20.9s	remaining: 30.1s
41:	learn: 7362.1108078	total: 21.7s	remaining: 29.9s
42:	learn: 7300.3860396	total: 21.9s	remaining: 29s
43:	learn: 7238.7135201	total: 2

73:	learn: 6298.7355849	total: 7.75s	remaining: 1m 37s
74:	learn: 6282.4245449	total: 7.87s	remaining: 1m 37s
75:	learn: 6268.9340452	total: 7.92s	remaining: 1m 36s
76:	learn: 6253.8185836	total: 8.01s	remaining: 1m 36s
77:	learn: 6239.4366290	total: 8.11s	remaining: 1m 35s
78:	learn: 6226.3487995	total: 8.24s	remaining: 1m 36s
79:	learn: 6215.4947066	total: 8.29s	remaining: 1m 35s
80:	learn: 6202.0554486	total: 8.37s	remaining: 1m 34s
81:	learn: 6189.6988149	total: 8.47s	remaining: 1m 34s
82:	learn: 6180.2962079	total: 8.59s	remaining: 1m 34s
83:	learn: 6168.1189533	total: 8.72s	remaining: 1m 35s
84:	learn: 6157.4410617	total: 8.83s	remaining: 1m 35s
85:	learn: 6149.9400321	total: 8.89s	remaining: 1m 34s
86:	learn: 6140.9563856	total: 9.02s	remaining: 1m 34s
87:	learn: 6132.2598990	total: 9.14s	remaining: 1m 34s
88:	learn: 6124.3632308	total: 9.25s	remaining: 1m 34s
89:	learn: 6116.6217995	total: 9.35s	remaining: 1m 34s
90:	learn: 6110.4293102	total: 9.46s	remaining: 1m 34s
91:	learn:

223:	learn: 5869.2315690	total: 23.6s	remaining: 1m 21s
224:	learn: 5869.0158346	total: 23.6s	remaining: 1m 21s
225:	learn: 5868.0603300	total: 23.7s	remaining: 1m 21s
226:	learn: 5867.7913966	total: 23.8s	remaining: 1m 21s
227:	learn: 5866.5190839	total: 24s	remaining: 1m 21s
228:	learn: 5866.2880629	total: 24.1s	remaining: 1m 21s
229:	learn: 5865.9687801	total: 24.2s	remaining: 1m 20s
230:	learn: 5865.1552520	total: 24.3s	remaining: 1m 20s
231:	learn: 5864.3882140	total: 24.4s	remaining: 1m 20s
232:	learn: 5863.2119154	total: 24.5s	remaining: 1m 20s
233:	learn: 5862.9481927	total: 24.6s	remaining: 1m 20s
234:	learn: 5862.9393732	total: 24.6s	remaining: 1m 20s
235:	learn: 5862.5912941	total: 24.7s	remaining: 1m 19s
236:	learn: 5862.5804985	total: 24.7s	remaining: 1m 19s
237:	learn: 5862.4731251	total: 24.7s	remaining: 1m 19s
238:	learn: 5861.5306592	total: 24.9s	remaining: 1m 19s
239:	learn: 5861.5247988	total: 24.9s	remaining: 1m 18s
240:	learn: 5861.2385440	total: 25s	remaining: 1m 

372:	learn: 5815.1325674	total: 37.8s	remaining: 1m 3s
373:	learn: 5814.4695938	total: 38s	remaining: 1m 3s
374:	learn: 5814.1633487	total: 38.1s	remaining: 1m 3s
375:	learn: 5814.0877403	total: 38.2s	remaining: 1m 3s
376:	learn: 5813.8675035	total: 38.3s	remaining: 1m 3s
377:	learn: 5813.8130957	total: 38.4s	remaining: 1m 3s
378:	learn: 5813.2687525	total: 38.6s	remaining: 1m 3s
379:	learn: 5813.0247011	total: 38.7s	remaining: 1m 3s
380:	learn: 5812.7994669	total: 38.8s	remaining: 1m 3s
381:	learn: 5812.6131268	total: 38.9s	remaining: 1m 2s
382:	learn: 5812.3548303	total: 39s	remaining: 1m 2s
383:	learn: 5811.8204576	total: 39.2s	remaining: 1m 2s
384:	learn: 5811.4459375	total: 39.3s	remaining: 1m 2s
385:	learn: 5811.1475449	total: 39.4s	remaining: 1m 2s
386:	learn: 5810.9468729	total: 39.5s	remaining: 1m 2s
387:	learn: 5810.6004326	total: 39.6s	remaining: 1m 2s
388:	learn: 5810.1379475	total: 39.8s	remaining: 1m 2s
389:	learn: 5809.3560317	total: 39.9s	remaining: 1m 2s
390:	learn: 58

...

9398:	learn: 4757.0634511	total: 2h 12m 25s	remaining: 8m 28s
9399:	learn: 4757.0449951	total: 2h 12m 26s	remaining: 8m 27s
9400:	learn: 4757.0167565	total: 2h 12m 27s	remaining: 8m 26s
9401:	learn: 4757.0160612	total: 2h 12m 28s	remaining: 8m 25s
9402:	learn: 4757.0159277	total: 2h 12m 28s	remaining: 8m 24s
9403:	learn: 4756.9595900	total: 2h 12m 29s	remaining: 8m 23s
9404:	learn: 4756.9303691	total: 2h 12m 30s	remaining: 8m 23s
9405:	learn: 4756.9303428	total: 2h 12m 31s	remaining: 8m 22s
9406:	learn: 4756.7855727	total: 2h 12m 32s	remaining: 8m 21s
9407:	learn: 4756.7727735	total: 2h 12m 33s	remaining: 8m 20s
9408:	learn: 4756.7692710	total: 2h 12m 34s	remaining: 8m 19s
9409:	learn: 4756.7691402	total: 2h 12m 35s	remaining: 8m 18s
9410:	learn: 4756.7668049	total: 2h 12m 36s	remaining: 8m 17s
9411:	learn: 4756.7667435	total: 2h 12m 37s	remaining: 8m 17s
9412:	learn: 4756.7267170	total: 2h 12m 38s	remaining: 8m 16s
9413:	learn: 4756.7266878	total: 2h 12m 39s	remaining: 8m 15s
9414:	le

9532:	learn: 4750.9301051	total: 2h 14m 24s	remaining: 6m 35s
9533:	learn: 4750.9297958	total: 2h 14m 25s	remaining: 6m 34s
9534:	learn: 4750.9157912	total: 2h 14m 26s	remaining: 6m 33s
9535:	learn: 4750.8956053	total: 2h 14m 27s	remaining: 6m 32s
9536:	learn: 4750.8953880	total: 2h 14m 28s	remaining: 6m 31s
9537:	learn: 4750.8952440	total: 2h 14m 29s	remaining: 6m 30s
9538:	learn: 4750.8662031	total: 2h 14m 29s	remaining: 6m 30s
9539:	learn: 4750.8661692	total: 2h 14m 30s	remaining: 6m 29s
9540:	learn: 4750.5194802	total: 2h 14m 31s	remaining: 6m 28s
9541:	learn: 4750.5183895	total: 2h 14m 32s	remaining: 6m 27s
9542:	learn: 4750.5161171	total: 2h 14m 33s	remaining: 6m 26s
9543:	learn: 4750.3550881	total: 2h 14m 34s	remaining: 6m 25s
9544:	learn: 4750.3424437	total: 2h 14m 35s	remaining: 6m 24s
9545:	learn: 4750.3259049	total: 2h 14m 36s	remaining: 6m 24s
9546:	learn: 4750.0534273	total: 2h 14m 37s	remaining: 6m 23s
9547:	learn: 4750.0384200	total: 2h 14m 38s	remaining: 6m 22s
9548:	le

9666:	learn: 4744.2840904	total: 2h 16m 25s	remaining: 4m 41s
9667:	learn: 4744.1456175	total: 2h 16m 25s	remaining: 4m 41s
9668:	learn: 4744.1325237	total: 2h 16m 26s	remaining: 4m 40s
9669:	learn: 4744.1279625	total: 2h 16m 27s	remaining: 4m 39s
9670:	learn: 4744.1279146	total: 2h 16m 28s	remaining: 4m 38s
9671:	learn: 4743.8692501	total: 2h 16m 29s	remaining: 4m 37s
9672:	learn: 4743.8687618	total: 2h 16m 30s	remaining: 4m 36s
9673:	learn: 4743.7358315	total: 2h 16m 31s	remaining: 4m 36s
9674:	learn: 4743.7354692	total: 2h 16m 32s	remaining: 4m 35s
9675:	learn: 4743.6600726	total: 2h 16m 33s	remaining: 4m 34s
9676:	learn: 4743.3781375	total: 2h 16m 34s	remaining: 4m 33s
9677:	learn: 4743.3779957	total: 2h 16m 35s	remaining: 4m 32s
9678:	learn: 4743.3779098	total: 2h 16m 36s	remaining: 4m 31s
9679:	learn: 4743.3620435	total: 2h 16m 37s	remaining: 4m 30s
9680:	learn: 4743.3583303	total: 2h 16m 38s	remaining: 4m 30s
9681:	learn: 4743.3181557	total: 2h 16m 39s	remaining: 4m 29s
9682:	le

9800:	learn: 4736.5115955	total: 2h 18m 31s	remaining: 2m 48s
9801:	learn: 4736.5098133	total: 2h 18m 31s	remaining: 2m 47s
9802:	learn: 4736.5098024	total: 2h 18m 31s	remaining: 2m 47s
9803:	learn: 4736.5097877	total: 2h 18m 32s	remaining: 2m 46s
9804:	learn: 4736.3701759	total: 2h 18m 33s	remaining: 2m 45s
9805:	learn: 4736.3653814	total: 2h 18m 34s	remaining: 2m 44s
9806:	learn: 4736.3653544	total: 2h 18m 35s	remaining: 2m 43s
9807:	learn: 4736.3584004	total: 2h 18m 36s	remaining: 2m 42s
9808:	learn: 4736.1229573	total: 2h 18m 37s	remaining: 2m 41s
9809:	learn: 4736.1177079	total: 2h 18m 38s	remaining: 2m 41s
9810:	learn: 4736.0550309	total: 2h 18m 39s	remaining: 2m 40s
9811:	learn: 4735.8796920	total: 2h 18m 40s	remaining: 2m 39s
9812:	learn: 4735.8787108	total: 2h 18m 41s	remaining: 2m 38s
9813:	learn: 4735.8729271	total: 2h 18m 42s	remaining: 2m 37s
9814:	learn: 4735.8722775	total: 2h 18m 43s	remaining: 2m 36s
9815:	learn: 4735.8706709	total: 2h 18m 44s	remaining: 2m 36s
9816:	le

9934:	learn: 4727.6114360	total: 2h 20m 37s	remaining: 55.2s
9935:	learn: 4727.4711961	total: 2h 20m 38s	remaining: 54.4s
9936:	learn: 4727.4666143	total: 2h 20m 39s	remaining: 53.5s
9937:	learn: 4727.4624049	total: 2h 20m 40s	remaining: 52.7s
9938:	learn: 4727.4545747	total: 2h 20m 41s	remaining: 51.8s
9939:	learn: 4727.4428741	total: 2h 20m 42s	remaining: 51s
9940:	learn: 4727.4393138	total: 2h 20m 43s	remaining: 50.1s
9941:	learn: 4727.4260477	total: 2h 20m 44s	remaining: 49.3s
9942:	learn: 4727.4249267	total: 2h 20m 45s	remaining: 48.4s
9943:	learn: 4727.4232040	total: 2h 20m 46s	remaining: 47.6s
9944:	learn: 4727.4182945	total: 2h 20m 47s	remaining: 46.7s
9945:	learn: 4727.4181623	total: 2h 20m 48s	remaining: 45.9s
9946:	learn: 4727.4180561	total: 2h 20m 48s	remaining: 45s
9947:	learn: 4727.3940980	total: 2h 20m 49s	remaining: 44.2s
9948:	learn: 4727.3925365	total: 2h 20m 50s	remaining: 43.3s
9949:	learn: 4727.3924753	total: 2h 20m 51s	remaining: 42.5s
9950:	learn: 4727.1472366	to

## Model Selection

In [49]:
pprint(models)

{'1 depth': (<catboost.core.CatBoostRegressor object at 0x000002851FA295F8>,
             0.3532877815470279),
 '10 iterations and 10 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520BA6128>,
                                -1.5472992634588265),
 '10 iterations and 6 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520BA69E8>,
                               -1.549647361507963),
 '100 iter': (<catboost.core.CatBoostRegressor object at 0x0000028520BA6438>,
              0.33617443710842654),
 '100 iterations and 10 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520C0B978>,
                                 0.3379910766681695),
 '100 iterations and 6 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520C0BBA8>,
                                0.3342404853308497),
 '1000 iterations and 10 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520C0BC50>,
                                  0.3869155316163564),
 '1000 iterations and 6 d

In [50]:
mdl = models['10000 iterations and 6 depth'][0]

In [51]:
fi = mdl.get_feature_importance()
for i in range(len(X_train.columns)):
    print(X_train.columns[i], '\t', fi[i])

pay_ub92 	 8.122388712323518
age 	 10.957189268486369
sex 	 2.218493683261131
provider 	 11.021349911222153
moa 	 2.0948789893384565
yoa 	 7.795588912742114
admtype 	 20.856298415380845
asource 	 4.692535073798569
pt_state 	 2.0558460504276224
diag_adm 	 4.4791789468507055
campus 	 4.591567764392483
er_mode 	 3.4176248432187943
race 	 2.0402150293440418
diag_cat3 	 13.003495022955198
diag_cat4 	 2.653349376258012


In [56]:
X_train2 = X_train.drop(['diag_cat3', 'diag_cat4'], axis='columns')
X_test2 = X_test.drop(['diag_cat3', 'diag_cat4'], axis='columns')
cat_features2 = [0, 2, 3, 4, 6, 7]
cat_features2.extend(range(8, len(X_train2.columns)))
print(cat_features2)
print(X_train.columns[cat_features2])

[0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12]
Index(['pay_ub92', 'sex', 'provider', 'moa', 'admtype', 'asource', 'pt_state',
       'diag_adm', 'campus', 'er_mode', 'race'],
      dtype='object')


In [59]:
m, s = fit_model({'iterations':1000}, X_train=X_train2, X_test=X_test2, cat_features=cat_features2)
model_name = '1000 iter no diag_cat'
models[model_name] = m, s

0:	learn: 14619.0981227	total: 73.1ms	remaining: 1m 12s
1:	learn: 14272.7949043	total: 111ms	remaining: 55.5s
2:	learn: 13935.3101337	total: 212ms	remaining: 1m 10s
3:	learn: 13613.0196720	total: 266ms	remaining: 1m 6s
4:	learn: 13300.2106265	total: 385ms	remaining: 1m 16s
5:	learn: 12994.8705445	total: 525ms	remaining: 1m 26s
6:	learn: 12705.4287534	total: 647ms	remaining: 1m 31s
7:	learn: 12427.7379231	total: 733ms	remaining: 1m 30s
8:	learn: 12159.0317021	total: 819ms	remaining: 1m 30s
9:	learn: 11899.0784516	total: 978ms	remaining: 1m 36s
10:	learn: 11650.7403575	total: 1.08s	remaining: 1m 37s
11:	learn: 11414.6832622	total: 1.16s	remaining: 1m 35s
12:	learn: 11182.1511647	total: 1.25s	remaining: 1m 34s
13:	learn: 10964.6104287	total: 1.34s	remaining: 1m 34s
14:	learn: 10750.6174072	total: 1.48s	remaining: 1m 36s
15:	learn: 10545.1013789	total: 1.56s	remaining: 1m 36s
16:	learn: 10346.0325340	total: 1.65s	remaining: 1m 35s
17:	learn: 10153.7635556	total: 1.76s	remaining: 1m 36s
18:

152:	learn: 5954.8630780	total: 14.2s	remaining: 1m 18s
153:	learn: 5952.8292255	total: 14.3s	remaining: 1m 18s
154:	learn: 5952.0424971	total: 14.5s	remaining: 1m 18s
155:	learn: 5950.3669237	total: 14.6s	remaining: 1m 18s
156:	learn: 5949.2899547	total: 14.8s	remaining: 1m 19s
157:	learn: 5947.7876775	total: 14.9s	remaining: 1m 19s
158:	learn: 5946.8944743	total: 15s	remaining: 1m 19s
159:	learn: 5946.2011764	total: 15s	remaining: 1m 19s
160:	learn: 5945.3912884	total: 15.1s	remaining: 1m 18s
161:	learn: 5945.1552571	total: 15.2s	remaining: 1m 18s
162:	learn: 5942.9621027	total: 15.4s	remaining: 1m 18s
163:	learn: 5941.6221137	total: 15.5s	remaining: 1m 18s
164:	learn: 5941.2893944	total: 15.6s	remaining: 1m 18s
165:	learn: 5941.0924007	total: 15.6s	remaining: 1m 18s
166:	learn: 5939.7523541	total: 15.7s	remaining: 1m 18s
167:	learn: 5939.1149632	total: 15.8s	remaining: 1m 18s
168:	learn: 5938.0924626	total: 15.9s	remaining: 1m 18s
169:	learn: 5936.7036476	total: 16s	remaining: 1m 18

300:	learn: 5876.2023390	total: 27.1s	remaining: 1m 2s
301:	learn: 5876.2023276	total: 27.1s	remaining: 1m 2s
302:	learn: 5875.4788028	total: 27.2s	remaining: 1m 2s
303:	learn: 5875.4633723	total: 27.3s	remaining: 1m 2s
304:	learn: 5875.4633636	total: 27.3s	remaining: 1m 2s
305:	learn: 5875.3698910	total: 27.4s	remaining: 1m 2s
306:	learn: 5875.3549160	total: 27.5s	remaining: 1m 2s
307:	learn: 5875.3151777	total: 27.5s	remaining: 1m 1s
308:	learn: 5875.2833989	total: 27.6s	remaining: 1m 1s
309:	learn: 5874.3158469	total: 27.7s	remaining: 1m 1s
310:	learn: 5873.7020031	total: 27.9s	remaining: 1m 1s
311:	learn: 5873.4099458	total: 28s	remaining: 1m 1s
312:	learn: 5873.3186499	total: 28.1s	remaining: 1m 1s
313:	learn: 5873.3186448	total: 28.1s	remaining: 1m 1s
314:	learn: 5873.3186401	total: 28.1s	remaining: 1m 1s
315:	learn: 5872.8225842	total: 28.2s	remaining: 1m 1s
316:	learn: 5872.1880595	total: 28.3s	remaining: 1m 1s
317:	learn: 5871.8455761	total: 28.4s	remaining: 1m
318:	learn: 587

452:	learn: 5829.3021914	total: 41.9s	remaining: 50.6s
453:	learn: 5828.8630070	total: 42s	remaining: 50.6s
454:	learn: 5828.6271247	total: 42.1s	remaining: 50.5s
455:	learn: 5828.2352924	total: 42.2s	remaining: 50.4s
456:	learn: 5828.1225034	total: 42.3s	remaining: 50.3s
457:	learn: 5827.7608534	total: 42.4s	remaining: 50.2s
458:	learn: 5827.6662521	total: 42.5s	remaining: 50.1s
459:	learn: 5827.4668422	total: 42.6s	remaining: 50.1s
460:	learn: 5827.3607635	total: 42.7s	remaining: 50s
461:	learn: 5826.7289001	total: 42.9s	remaining: 49.9s
462:	learn: 5826.3712912	total: 43s	remaining: 49.8s
463:	learn: 5825.8575753	total: 43.1s	remaining: 49.7s
464:	learn: 5825.4573515	total: 43.2s	remaining: 49.7s
465:	learn: 5825.3344291	total: 43.3s	remaining: 49.6s
466:	learn: 5825.1408830	total: 43.4s	remaining: 49.5s
467:	learn: 5825.1370916	total: 43.4s	remaining: 49.4s
468:	learn: 5824.9016361	total: 43.5s	remaining: 49.3s
469:	learn: 5824.5579882	total: 43.6s	remaining: 49.2s
470:	learn: 5824

604:	learn: 5802.6133791	total: 56.8s	remaining: 37.1s
605:	learn: 5802.3370017	total: 56.9s	remaining: 37s
606:	learn: 5802.2891847	total: 57.1s	remaining: 36.9s
607:	learn: 5801.9544251	total: 57.2s	remaining: 36.8s
608:	learn: 5801.6777621	total: 57.3s	remaining: 36.8s
609:	learn: 5801.5757246	total: 57.4s	remaining: 36.7s
610:	learn: 5801.5602011	total: 57.4s	remaining: 36.6s
611:	learn: 5801.0859052	total: 57.5s	remaining: 36.5s
612:	learn: 5800.4917975	total: 57.6s	remaining: 36.4s
613:	learn: 5799.8340819	total: 57.8s	remaining: 36.3s
614:	learn: 5799.8183526	total: 57.9s	remaining: 36.2s
615:	learn: 5799.5152254	total: 58s	remaining: 36.1s
616:	learn: 5799.4900633	total: 58.1s	remaining: 36s
617:	learn: 5799.4765204	total: 58.2s	remaining: 35.9s
618:	learn: 5799.2442073	total: 58.3s	remaining: 35.9s
619:	learn: 5799.2290999	total: 58.3s	remaining: 35.7s
620:	learn: 5799.2204656	total: 58.4s	remaining: 35.6s
621:	learn: 5799.1036705	total: 58.5s	remaining: 35.5s
622:	learn: 5799

755:	learn: 5780.1706541	total: 1m 11s	remaining: 23.1s
756:	learn: 5780.0233496	total: 1m 11s	remaining: 23s
757:	learn: 5780.0135174	total: 1m 11s	remaining: 22.9s
758:	learn: 5779.9513862	total: 1m 11s	remaining: 22.8s
759:	learn: 5779.8133601	total: 1m 11s	remaining: 22.7s
760:	learn: 5779.7986962	total: 1m 11s	remaining: 22.6s
761:	learn: 5779.6792847	total: 1m 12s	remaining: 22.5s
762:	learn: 5779.6520869	total: 1m 12s	remaining: 22.4s
763:	learn: 5779.5623557	total: 1m 12s	remaining: 22.3s
764:	learn: 5779.5430167	total: 1m 12s	remaining: 22.2s
765:	learn: 5779.4038678	total: 1m 12s	remaining: 22.1s
766:	learn: 5779.2197579	total: 1m 12s	remaining: 22s
767:	learn: 5778.8127870	total: 1m 12s	remaining: 21.9s
768:	learn: 5778.8127870	total: 1m 12s	remaining: 21.8s
769:	learn: 5778.8027481	total: 1m 12s	remaining: 21.7s
770:	learn: 5778.7720418	total: 1m 12s	remaining: 21.6s
771:	learn: 5778.7330274	total: 1m 12s	remaining: 21.5s
772:	learn: 5778.7232500	total: 1m 12s	remaining: 21

902:	learn: 5760.6168574	total: 1m 25s	remaining: 9.23s
903:	learn: 5760.6055927	total: 1m 26s	remaining: 9.14s
904:	learn: 5760.6024156	total: 1m 26s	remaining: 9.04s
905:	learn: 5760.5193990	total: 1m 26s	remaining: 8.94s
906:	learn: 5760.1722606	total: 1m 26s	remaining: 8.85s
907:	learn: 5760.1722606	total: 1m 26s	remaining: 8.74s
908:	learn: 5760.0744464	total: 1m 26s	remaining: 8.65s
909:	learn: 5760.0061675	total: 1m 26s	remaining: 8.55s
910:	learn: 5759.9464912	total: 1m 26s	remaining: 8.46s
911:	learn: 5759.9421913	total: 1m 26s	remaining: 8.36s
912:	learn: 5759.5988543	total: 1m 26s	remaining: 8.27s
913:	learn: 5759.5779486	total: 1m 26s	remaining: 8.17s
914:	learn: 5759.5376943	total: 1m 26s	remaining: 8.07s
915:	learn: 5759.2381885	total: 1m 27s	remaining: 7.98s
916:	learn: 5758.8483150	total: 1m 27s	remaining: 7.89s
917:	learn: 5758.8430020	total: 1m 27s	remaining: 7.79s
918:	learn: 5758.8321749	total: 1m 27s	remaining: 7.7s
919:	learn: 5758.8313666	total: 1m 27s	remaining:

In [61]:
m, s = fit_model({'iterations':10000}, X_train=X_train2, X_test=X_test2, cat_features=cat_features2)
model_name = '10000 iter no diag_cat'
models[model_name] = m, s

0:	learn: 14619.1079407	total: 82.1ms	remaining: 13m 41s
1:	learn: 14272.1293275	total: 134ms	remaining: 11m 11s
2:	learn: 13939.8445598	total: 171ms	remaining: 9m 29s
3:	learn: 13615.6936049	total: 264ms	remaining: 11m
4:	learn: 13304.3531375	total: 342ms	remaining: 11m 22s
5:	learn: 13005.6706310	total: 384ms	remaining: 10m 39s
6:	learn: 12714.2306002	total: 437ms	remaining: 10m 23s
7:	learn: 12433.8666098	total: 545ms	remaining: 11m 20s
8:	learn: 12164.7024751	total: 597ms	remaining: 11m 3s
9:	learn: 11905.1489087	total: 696ms	remaining: 11m 34s
10:	learn: 11655.1499447	total: 790ms	remaining: 11m 57s
11:	learn: 11415.7044943	total: 858ms	remaining: 11m 54s
12:	learn: 11183.7630418	total: 973ms	remaining: 12m 27s
13:	learn: 10961.8539906	total: 1.02s	remaining: 12m 10s
14:	learn: 10749.6589588	total: 1.13s	remaining: 12m 28s
15:	learn: 10545.8894987	total: 1.23s	remaining: 12m 49s
16:	learn: 10351.0556542	total: 1.31s	remaining: 12m 47s
17:	learn: 10160.8232680	total: 1.39s	remainin

146:	learn: 5957.6117580	total: 13.7s	remaining: 15m 21s
147:	learn: 5956.1612265	total: 13.8s	remaining: 15m 20s
148:	learn: 5954.6432334	total: 13.9s	remaining: 15m 21s
149:	learn: 5954.3175964	total: 14s	remaining: 15m 17s
150:	learn: 5953.3501265	total: 14.1s	remaining: 15m 18s
151:	learn: 5952.7396564	total: 14.2s	remaining: 15m 17s
152:	learn: 5951.9032441	total: 14.2s	remaining: 15m 13s
153:	learn: 5950.5882887	total: 14.3s	remaining: 15m 13s
154:	learn: 5949.6697468	total: 14.4s	remaining: 15m 12s
155:	learn: 5948.1417572	total: 14.5s	remaining: 15m 12s
156:	learn: 5947.9082203	total: 14.5s	remaining: 15m 9s
157:	learn: 5946.5435024	total: 14.6s	remaining: 15m 9s
158:	learn: 5945.1754581	total: 14.7s	remaining: 15m 9s
159:	learn: 5944.4767943	total: 14.8s	remaining: 15m 10s
160:	learn: 5943.5492083	total: 14.9s	remaining: 15m 11s
161:	learn: 5942.4607314	total: 15s	remaining: 15m 10s
162:	learn: 5942.3070885	total: 15s	remaining: 15m 6s
163:	learn: 5940.7510958	total: 15.1s	rem

292:	learn: 5879.0670999	total: 25.5s	remaining: 14m 3s
293:	learn: 5879.0636584	total: 25.5s	remaining: 14m 2s
294:	learn: 5879.0636427	total: 25.5s	remaining: 14m
295:	learn: 5878.7605358	total: 25.6s	remaining: 14m
296:	learn: 5878.6091917	total: 25.8s	remaining: 14m 1s
297:	learn: 5877.7238716	total: 25.9s	remaining: 14m 1s
298:	learn: 5877.7238583	total: 25.9s	remaining: 13m 59s
299:	learn: 5877.5640458	total: 25.9s	remaining: 13m 58s
300:	learn: 5877.0504440	total: 26s	remaining: 13m 58s
301:	learn: 5876.5722394	total: 26.1s	remaining: 13m 59s
302:	learn: 5876.2065129	total: 26.2s	remaining: 13m 58s
303:	learn: 5875.5936286	total: 26.3s	remaining: 13m 59s
304:	learn: 5874.9732572	total: 26.4s	remaining: 14m
305:	learn: 5874.7758635	total: 26.5s	remaining: 14m
306:	learn: 5874.7734828	total: 26.6s	remaining: 13m 59s
307:	learn: 5874.4091172	total: 26.7s	remaining: 13m 59s
308:	learn: 5874.0932574	total: 26.8s	remaining: 14m
309:	learn: 5874.0595527	total: 26.8s	remaining: 13m 58s


438:	learn: 5834.3630919	total: 39s	remaining: 14m 9s
439:	learn: 5834.1591441	total: 39.1s	remaining: 14m 10s
440:	learn: 5833.3566000	total: 39.2s	remaining: 14m 10s
441:	learn: 5833.1002845	total: 39.3s	remaining: 14m 9s
442:	learn: 5832.6733475	total: 39.4s	remaining: 14m 9s
443:	learn: 5832.5937326	total: 39.5s	remaining: 14m 9s
444:	learn: 5832.5094688	total: 39.6s	remaining: 14m 10s
445:	learn: 5832.1214082	total: 39.7s	remaining: 14m 10s
446:	learn: 5831.8941608	total: 39.8s	remaining: 14m 11s
447:	learn: 5831.2578045	total: 39.9s	remaining: 14m 11s
448:	learn: 5831.1172267	total: 40s	remaining: 14m 11s
449:	learn: 5830.9474843	total: 40.1s	remaining: 14m 11s
450:	learn: 5830.6468048	total: 40.2s	remaining: 14m 11s
451:	learn: 5830.2912812	total: 40.3s	remaining: 14m 11s
452:	learn: 5829.9285891	total: 40.4s	remaining: 14m 11s
453:	learn: 5829.9232942	total: 40.5s	remaining: 14m 11s
454:	learn: 5829.7681819	total: 40.6s	remaining: 14m 11s
455:	learn: 5829.3342353	total: 40.7s	r

584:	learn: 5802.5405652	total: 53.7s	remaining: 14m 23s
585:	learn: 5802.1840700	total: 53.8s	remaining: 14m 23s
586:	learn: 5801.9985666	total: 53.9s	remaining: 14m 23s
587:	learn: 5801.8019838	total: 54s	remaining: 14m 23s
588:	learn: 5801.6649917	total: 54.1s	remaining: 14m 23s
589:	learn: 5801.2422973	total: 54.2s	remaining: 14m 23s
590:	learn: 5800.9716968	total: 54.3s	remaining: 14m 23s
591:	learn: 5800.8873328	total: 54.4s	remaining: 14m 23s
592:	learn: 5800.8426586	total: 54.5s	remaining: 14m 23s
593:	learn: 5800.6101922	total: 54.6s	remaining: 14m 23s
594:	learn: 5800.3264002	total: 54.7s	remaining: 14m 24s
595:	learn: 5800.1143979	total: 54.8s	remaining: 14m 24s
596:	learn: 5799.8943711	total: 54.9s	remaining: 14m 23s
597:	learn: 5799.8276440	total: 54.9s	remaining: 14m 23s
598:	learn: 5799.7088385	total: 55s	remaining: 14m 23s
599:	learn: 5799.6774869	total: 55.1s	remaining: 14m 23s
600:	learn: 5799.5212196	total: 55.2s	remaining: 14m 23s
601:	learn: 5799.3910029	total: 55.

...

9375:	learn: 5306.8918147	total: 15m 32s	remaining: 1m 2s
9376:	learn: 5306.8900677	total: 15m 32s	remaining: 1m 1s
9377:	learn: 5306.8821904	total: 15m 32s	remaining: 1m 1s
9378:	learn: 5306.8633823	total: 15m 32s	remaining: 1m 1s
9379:	learn: 5306.8630294	total: 15m 32s	remaining: 1m 1s
9380:	learn: 5306.8145697	total: 15m 32s	remaining: 1m 1s
9381:	learn: 5306.8144137	total: 15m 33s	remaining: 1m 1s
9382:	learn: 5306.6846897	total: 15m 33s	remaining: 1m 1s
9383:	learn: 5306.6525494	total: 15m 33s	remaining: 1m 1s
9384:	learn: 5306.6255781	total: 15m 33s	remaining: 1m 1s
9385:	learn: 5306.6240498	total: 15m 33s	remaining: 1m 1s
9386:	learn: 5306.6227416	total: 15m 33s	remaining: 1m
9387:	learn: 5306.3770975	total: 15m 33s	remaining: 1m
9388:	learn: 5306.2801293	total: 15m 33s	remaining: 1m
9389:	learn: 5306.2800934	total: 15m 33s	remaining: 1m
9390:	learn: 5306.2674000	total: 15m 33s	remaining: 1m
9391:	learn: 5306.2630432	total: 15m 34s	remaining: 1m
9392:	learn: 5306.0975240	total:

9518:	learn: 5302.3445272	total: 15m 46s	remaining: 47.8s
9519:	learn: 5302.3094933	total: 15m 46s	remaining: 47.7s
9520:	learn: 5302.3090999	total: 15m 46s	remaining: 47.6s
9521:	learn: 5302.3088914	total: 15m 46s	remaining: 47.5s
9522:	learn: 5302.3064722	total: 15m 46s	remaining: 47.4s
9523:	learn: 5302.3063158	total: 15m 46s	remaining: 47.3s
9524:	learn: 5302.2400836	total: 15m 46s	remaining: 47.2s
9525:	learn: 5302.0849900	total: 15m 46s	remaining: 47.1s
9526:	learn: 5302.0498435	total: 15m 46s	remaining: 47s
9527:	learn: 5302.0430517	total: 15m 46s	remaining: 46.9s
9528:	learn: 5302.0426924	total: 15m 46s	remaining: 46.8s
9529:	learn: 5302.0373207	total: 15m 47s	remaining: 46.7s
9530:	learn: 5302.0227436	total: 15m 47s	remaining: 46.6s
9531:	learn: 5302.0223009	total: 15m 47s	remaining: 46.5s
9532:	learn: 5301.9302302	total: 15m 47s	remaining: 46.4s
9533:	learn: 5301.8885819	total: 15m 47s	remaining: 46.3s
9534:	learn: 5301.8302358	total: 15m 47s	remaining: 46.2s
9535:	learn: 530

9660:	learn: 5297.2303865	total: 15m 59s	remaining: 33.7s
9661:	learn: 5297.2301485	total: 15m 59s	remaining: 33.6s
9662:	learn: 5297.2036976	total: 15m 59s	remaining: 33.5s
9663:	learn: 5297.2034429	total: 15m 59s	remaining: 33.4s
9664:	learn: 5297.1997378	total: 15m 59s	remaining: 33.3s
9665:	learn: 5297.1962094	total: 16m	remaining: 33.2s
9666:	learn: 5297.1961485	total: 16m	remaining: 33.1s
9667:	learn: 5297.1908151	total: 16m	remaining: 33s
9668:	learn: 5297.1222847	total: 16m	remaining: 32.9s
9669:	learn: 5297.1208055	total: 16m	remaining: 32.8s
9670:	learn: 5297.1205686	total: 16m	remaining: 32.7s
9671:	learn: 5297.0429047	total: 16m	remaining: 32.6s
9672:	learn: 5297.0428845	total: 16m	remaining: 32.5s
9673:	learn: 5296.9878266	total: 16m	remaining: 32.4s
9674:	learn: 5296.9852706	total: 16m	remaining: 32.3s
9675:	learn: 5296.9851314	total: 16m	remaining: 32.2s
9676:	learn: 5296.9845908	total: 16m 1s	remaining: 32.1s
9677:	learn: 5296.9840862	total: 16m 1s	remaining: 32s
9678:	

9806:	learn: 5293.6819128	total: 16m 13s	remaining: 19.2s
9807:	learn: 5293.6747954	total: 16m 13s	remaining: 19.1s
9808:	learn: 5293.6739990	total: 16m 13s	remaining: 19s
9809:	learn: 5293.6444242	total: 16m 13s	remaining: 18.9s
9810:	learn: 5293.5753599	total: 16m 13s	remaining: 18.8s
9811:	learn: 5293.5025883	total: 16m 13s	remaining: 18.7s
9812:	learn: 5293.4366273	total: 16m 14s	remaining: 18.6s
9813:	learn: 5293.4360745	total: 16m 14s	remaining: 18.5s
9814:	learn: 5293.4165295	total: 16m 14s	remaining: 18.4s
9815:	learn: 5293.4050592	total: 16m 14s	remaining: 18.3s
9816:	learn: 5293.1997165	total: 16m 14s	remaining: 18.2s
9817:	learn: 5293.1043932	total: 16m 14s	remaining: 18.1s
9818:	learn: 5293.0949177	total: 16m 14s	remaining: 18s
9819:	learn: 5293.0377731	total: 16m 14s	remaining: 17.9s
9820:	learn: 5292.9893930	total: 16m 14s	remaining: 17.8s
9821:	learn: 5292.9692407	total: 16m 14s	remaining: 17.7s
9822:	learn: 5292.9692149	total: 16m 14s	remaining: 17.6s
9823:	learn: 5292.

9948:	learn: 5288.6221036	total: 16m 26s	remaining: 5.06s
9949:	learn: 5288.5572399	total: 16m 27s	remaining: 4.96s
9950:	learn: 5288.5395947	total: 16m 27s	remaining: 4.86s
9951:	learn: 5288.5197211	total: 16m 27s	remaining: 4.76s
9952:	learn: 5288.4975985	total: 16m 27s	remaining: 4.66s
9953:	learn: 5288.4951296	total: 16m 27s	remaining: 4.56s
9954:	learn: 5288.4931047	total: 16m 27s	remaining: 4.46s
9955:	learn: 5288.4855810	total: 16m 27s	remaining: 4.36s
9956:	learn: 5288.4599768	total: 16m 27s	remaining: 4.26s
9957:	learn: 5288.4521271	total: 16m 27s	remaining: 4.17s
9958:	learn: 5288.2254187	total: 16m 27s	remaining: 4.07s
9959:	learn: 5288.2241791	total: 16m 28s	remaining: 3.97s
9960:	learn: 5288.1468230	total: 16m 28s	remaining: 3.87s
9961:	learn: 5288.1373273	total: 16m 28s	remaining: 3.77s
9962:	learn: 5288.1342793	total: 16m 28s	remaining: 3.67s
9963:	learn: 5288.0846465	total: 16m 28s	remaining: 3.57s
9964:	learn: 5288.0573356	total: 16m 28s	remaining: 3.47s
9965:	learn: 5

In [62]:
pprint(models)

{'1 depth': (<catboost.core.CatBoostRegressor object at 0x000002851FA295F8>,
             0.3532877815470279),
 '10 iterations and 10 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520BA6128>,
                                -1.5472992634588265),
 '10 iterations and 6 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520BA69E8>,
                               -1.549647361507963),
 '100 iter': (<catboost.core.CatBoostRegressor object at 0x0000028520BA6438>,
              0.33617443710842654),
 '100 iterations and 10 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520C0B978>,
                                 0.3379910766681695),
 '100 iterations and 6 depth': (<catboost.core.CatBoostRegressor object at 0x0000028520C0BBA8>,
                                0.3342404853308497),
 '1000 iter no diag_cat': (<catboost.core.CatBoostRegressor object at 0x0000028520BA65C0>,
                           0.37811747148301966),
 '1000 iterations and 10 depth': (<cat